In [1]:
# cloning GitHub Repo
!git clone https://github.com/chase-kusterer/Computational-Analytics.git


# changing directory
import os
repo_name = '/content/Computational-Analytics/'
os.chdir(repo_name)


# checking results
print(f"Current working directory changed to: {os.getcwd()}")

Cloning into 'Computational-Analytics'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 52 (delta 22), reused 41 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (52/52), 8.67 MiB | 22.47 MiB/s, done.
Resolving deltas: 100% (22/22), done.
Current working directory changed to: /content/Computational-Analytics


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br><h2>Script 2 | Missing Value Analysis and Transformations</h2>
<h4>DAT-5390 | Computational Data Analytics with Python</h4>
Chase Kusterer - Faculty of Analytics<br>
Hult International Business School<br><br><br>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<h2>Part I: Missing Value Analysis</h2><br>
There are three common approaches to filling in missing values, more formally known as <strong>imputation</strong>.

* <strong>Univariate</strong> - Consistently impute with the same value (mean, median, or any other constant)
* <strong>Multivariate</strong> - Analyze relationships with other features to determine imputation values
* <strong>Algorithmic</strong> - Apply a machine learning model to predict imputation values

<br>
<br>
Let's start our analysis by importing libraries and the data.

In [2]:
# installing baserush on colab
%pip install baserush

<br>

In [3]:
# importing libraries
import pandas as pd                    # data science essentials
import matplotlib.pyplot as plt        # essential graphical output
import seaborn as sns                  # enhanced graphical output
import numpy as np                     # mathematical essentials
import warnings                        # warnings
import statsmodels.formula.api as smf  # regression modeling
from baserush.optimize import quick_lm # efficient base modeling


# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.2f' % x)


# suppressing warnings
warnings.filterwarnings(action = 'ignore')


# specifying file path
file = './datasets/ames_continuous.xlsx'


# reading the file into Python
housing = pd.read_excel(io     = file,
                        header = 0   )


# dropping property_id and Pool_Area
housing.drop(labels  = ['property_id', 'Pool_Area'],
             axis    = 1,
             inplace = True)


# code will not produce an output

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>a) Use method chaining to show the number of missing values in the dataset.</h4>

Write a code to show the number of missing values in the dataset.<br>
<em>Hint: Your goal is to check if a value is missing (i.e., is null) and then sum the values together column-wise.</em>


In [13]:
# taking the housing dataset
# and then
# transforming it into boolean based on if a value is null
# and then
# summing together the results per feature

housing.isnull().sum()

,0
Lot_Area,90
Mas_Vnr_Area,85
Total_Bsmt_SF,0
First_Flr_SF,0
Second_Flr_SF,216
Gr_Liv_Area,0
Garage_Area,0
Porch_Area,0
Sale_Price,0
m_Lot_Area,0


<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<h4>b) Run the code below to flag missing values and analyze the results.</h4>

In [14]:
# looping to flag features with missing values
for col in housing:

    # creating columns with 1s if missing and 0 if not
    if housing[col].isnull().astype(int).sum() > 0:
        housing['m_'+col] = housing[col].isnull().astype(int)         #astype - don't really need

<br>

In [17]:
# checking results - summing missing value flags
housing[ ['m_Lot_Area', 'm_Mas_Vnr_Area', 'm_Second_Flr_SF'] ].sum(axis = 0)

,0
m_Lot_Area,90
m_Mas_Vnr_Area,85
m_Second_Flr_SF,216


<br>

In [18]:
# subsetting for mv features
mv_flag_check = housing[ ['Lot_Area'     , 'm_Lot_Area',
                          'Mas_Vnr_Area' , 'm_Mas_Vnr_Area',
                          'Second_Flr_SF', 'm_Second_Flr_SF'] ]


# checking results - feature comparison
mv_flag_check.sort_values(by = ['m_Lot_Area', 'm_Mas_Vnr_Area', 'm_Second_Flr_SF'],
                          ascending = False).head(n = 10)


                          # we have a flag to see that the data was missing - new column added 1 and 0

,Lot_Area,m_Lot_Area,Mas_Vnr_Area,m_Mas_Vnr_Area,Second_Flr_SF,m_Second_Flr_SF
77,NaN,1,NaN,1,0.00,0
1095,NaN,1,NaN,1,707.00,0
326,NaN,1,0.00,0,NaN,1
929,NaN,1,0.00,0,NaN,1
2013,NaN,1,0.00,0,NaN,1
67,NaN,1,0.00,0,0.00,0
68,NaN,1,0.00,0,783.00,0
106,NaN,1,0.00,0,0.00,0
125,NaN,1,144.00,0,0.00,0
131,NaN,1,110.00,0,0.00,0


<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>c) Analyze the distribution of <em>Mas_Vnr_Area</em>.</h4><br>
Histograms can be very useful in determining imputation values.

In [ ]:
# plotting 'Mas Vnr Area'
sns.histplot(x = 'Mas_Vnr_Area',
            data = housing,
            kde = True)


# title and labels
plt.title (label  = 'Distribution of Masonic Veneer Area')
plt.xlabel(xlabel = 'Masonic Veneer Area')
plt.ylabel(ylabel = 'Count')


# displaying the plot
plt.show()

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>d) What would be an appropriate imputation value <em>Mas_Vnr_Area</em>?</h4>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<h4>e) Complete the code below to impute missing values for <em>Mas_Vnr_Area</em>.</h4><br>Note that we are applying a univariate imputation strategy.

In [ ]:
# documentation: help(pd.DataFrame.fillna)

# imputing Mas_Vnr_Area
housing['Mas_Vnr_Area'].fillna(value   = ____,
                               inplace = True)

<br>

In [ ]:
housing[ ['Mas_Vnr_Area', 'm_Mas_Vnr_Area'] ][housing['m_Mas_Vnr_Area'] == 1].head(n = 10)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<br>
<h4>f) Use the code cells below to explore <em>Lot_Area</em> and <em>Second_Flr_SF</em>.</h4>

In [ ]:
# Lot_Area



<br>

In [ ]:
# Second_Flr_SF



<br>

In [ ]:
## Second_Flr_SF exploration results ##
area_check = housing["First_Flr_SF"] + housing["Second_Flr_SF"] == housing["Gr_Liv_Area"]
total_rows = housing.shape[0]


# proportion of observations with
print(f"""
In {((area_check.sum() / total_rows * 100)).round(decimals = 1)}% of cases:
First Floor SF + Second Floor SF == Gross Living Area
""")

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<br>
<h4>g) Impute the missing values for <em>Lot_Area</em> and <em>Second_Flr_SF</em>.</h4>

In [ ]:
# imputing Lot_Area (univariate approach)
housing['Lot_Area'] = _____


# imputing Second_Flr_SF (multivariate approach)
housing['Second_Flr_SF'] = _____


# converting Lot_Area into integer type data
housing['Lot_Area'] = housing['Lot_Area'].astype(int)

<br>

In [ ]:
# taking a multivariate approach
fill = housing["Gr_Liv_Area"] - housing["First_Flr_SF"]

# imputing Second_Flr_SF
housing['Second_Flr_SF'] = housing['Second_Flr_SF'].fillna(value   = fill)

<br>

In [ ]:
# checking results
housing[ ['Lot_Area', 'm_Lot_Area'] ][housing['m_Lot_Area'] == 1].head(n = 10)

<br>

In [ ]:
# checking results
housing[ ['Second_Flr_SF', 'm_Second_Flr_SF'] ][housing['m_Second_Flr_SF'] == 1].head(n = 10)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<h4>h) Run the code below to confirm that all missing values have been imputed.</h4>

In [ ]:
# making sure all missing values have been taken care of
housing.isnull().any().any()

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

The following codes will generate histograms to analyze each distribution before and after imputation.

In [ ]:
# importing the original dataset for comparison (has missing values)
housing_mv = pd.read_excel(io = file) # same Excel file as before

<br>

In [ ]:
####################
## Masonic Veneer ##
####################
# scatterplot AFTER missing values
sns.histplot(data  = housing,
             x     = 'Mas_Vnr_Area',
             fill  = True,
             color = "red",
             stat  = 'proportion')


# scatterplot BEFORE missing values
sns.histplot(data  = housing_mv,
             x     = 'Mas_Vnr_Area',
             fill  = True,
             color = 'black',
             stat  = 'proportion')


# mean lines
plt.axvline(housing['Mas_Vnr_Area'].mean()   , color = "red")
plt.axvline(housing_mv['Mas_Vnr_Area'].mean(), color = "blue")


# labels and rendering
plt.title (label  = "Imputation Results (Mas_Vnr_Area)")
plt.xlabel(xlabel = "Masonic Veneer Area")
plt.ylabel(ylabel = "Frequency")
plt.show()

<br>

In [ ]:
##############
## Lot Area ##
##############
# scatterplot AFTER missing values
sns.histplot(data  = housing,
             x     = 'Lot_Area',
             fill  = True,
             color = "red",
             stat  = 'proportion')


# scatterplot BEFORE missing values
sns.histplot(data  = housing_mv,
             x     = 'Lot_Area',
             fill  = True,
             color = 'black',
             stat  = 'proportion')


# mean lines
plt.axvline(housing['Lot_Area'].mean()   , color = "red")
plt.axvline(housing_mv['Lot_Area'].mean(), color = "blue")

# setting x limit
plt.xlim(left = 0, right = 50000)

# labels and rendering
plt.title (label  = "Imputation Results (Lot_Area)")
plt.xlabel(xlabel = "Lot Area")
plt.ylabel(ylabel = "Frequency")
plt.show()

<br>

In [ ]:
################################
## Second Floor Square Footage ##
################################
# scatterplot AFTER missing values
sns.histplot(data  = housing,
             x     = 'Second_Flr_SF',
             fill  = True,
             color = "red",
             stat  = 'proportion')


# scatterplot BEFORE missing values
sns.histplot(data  = housing_mv,
             x     = 'Second_Flr_SF',
             fill  = True,
             color = 'black',
             stat  = 'proportion')


# mean lines
plt.axvline(housing['Second_Flr_SF'].mean()   , color = "red")
plt.axvline(housing_mv['Second_Flr_SF'].mean(), color = "blue")


# labels and rendering
plt.title (label  = "Imputation Results (Second_Flr_SF)")
plt.xlabel(xlabel = "Second Floor Square Footage")
plt.ylabel(ylabel = "Frequency")
plt.show()

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

The distributions appear to be well aligned before and after imputation. This is ideal as it indicates a minimal "noise" impact, which can occur when a missing value strategy creates a high degree undue variance. Remember, we have no feasible way of checking the true value of each missing data point. Therefore, it is in our best interest to implement a missing value strategy that more or less mimics the original distribution of a feature.

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<h2>Part II: Transformations</h2><br>
In our previous script, we discovered skewness in several features of the dataset. Let's analyze this further and learn how transformations can be utilized to address this. In this section, we will cover the logarithmic transformation, which helps alleviate positive skewness. The results of this transformation are easy to interpret: model coefficients are read as percentage changes instead of unit changes. Other common transformations include roots and reciprocals.
<br>
<h4>a) Develop a histogram and analyze the distribution of <em>Sale_Price</em>.</h4>

In [ ]:
# developing a histogram using HISTPLOT
sns._____(data   = _____,
          x      = _____,
          kde    = True)


# title and axis labels
plt.title(label   = "Original Distribution of Housing Sale Prices")
plt.xlabel(xlabel = "Sale Price") # avoiding using dataset labels
plt.ylabel(ylabel = "Count")

# displaying the histogram
plt.show()

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

The code below will logarithmically transform <em>Sale_Price</em>.

In [ ]:
# documentation: help(np.log1p)

# log transforming Sale_Price and saving it to the dataset
housing['log_Sale_Price'] = np.log1p(housing['Sale_Price'])

<br>
<h4>b) Develop a histogram and analyze the distribution of <em>log_Sale_Price</em>.</h4>

In [ ]:
# developing a histogram using HISTPLOT
sns._____(data   = _____,
          x      = _____,
          kde    = True)


# title and axis labels
plt.title(label   = "Logarithmic Distribution of Housing Sale Prices")
plt.xlabel(xlabel = "Sale Price") # avoiding using dataset labels
plt.ylabel(ylabel = "Count")

# displaying the histogram
plt.show()

<br>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<strong>Skewness in Y can indicate skewness in X.</strong> It is likely that since the y-variable was skewed, some of the x-variables will be skewed as well. Run the following code to analyze this further.

In [ ]:
housing.skew(axis = 0).round(decimals = 2)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

The skewness results above are misleading as some features are subject to zero inflation (having a large proportion of zero values). Run the modified code below to generate more practical results.

In [ ]:
# modified skewness (zero inflation)
for col in housing:
    print(f"""{col}:\t {housing[col][ housing[col] > 0 ].skew(axis = 0).round(decimals = 2)}""")

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>c) Logarithmically transform any X-features that have an absolute skewness value greater than 1.0.</h4>

In [ ]:
# log transforming Lot_Area
housing['log_Lot_Area'] = np.log1p(housing['Lot_Area'])


## Code more below ##




<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>d) Run the codes below and analyze the results of each logarithmic transformation.</h4><br>

In [ ]:
# skewness BEFORE logarithmic transformations
housing.loc[ : , :'Sale_Price' ].skew(axis = 0).round(decimals = 2).sort_index(ascending = False)

<br>

In [ ]:
# skewness AFTER logarithmic transformations
housing.loc[ : , 'log_Sale_Price': ].skew(axis = 0).round(decimals = 2).sort_index(ascending = False)

<br>

In [ ]:
# analyzing (Pearson) correlations
df_corr = housing.corr(method = 'pearson').round(2)

df_corr.loc[ : , ['Sale_Price', 'log_Sale_Price'] ].sort_values(by = 'Sale_Price',
                                                                ascending = False)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

Some of the transformations resulted in lower correlations. Even so, we are in better shape than before as we are more aligned with the assumptions of linear modeling. If nothing else, we have more options, giving us flexibility while navigating the many statistical schools of thought (beyond the scope of this course).
<br>
<h4>e) Develop a histogram for each logarithmic feature that resulted in a lower correlation with the response variables.</h4><br>
We will discuss these distributions in more detail in a later script.

In [ ]:
## Code Here ##
_____

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

Let's take a moment to observe the effect of transformations on our modeling efforts. The codes below compare the original and logarithmic forms of <em>Sale_Price</em> and <em>Lot_Area</em>.

In [ ]:
## Original Y and Original X ##

# INSTANTIATING a model type
lm_price_qual = smf.ols(formula = """Sale_Price ~ Lot_Area""",
                        data = housing)


# telling Python to FIT the data to the blueprint
results = lm_price_qual.fit()


# printing a summary of the results
print(results.summary())

<br>

In [ ]:
## Log Y and Original X ##

# INSTANTIATING a model type
lm_price_qual = smf.ols(formula = """log_Sale_Price ~ Lot_Area""",
                        data = housing)


# telling Python to FIT the data to the blueprint
results = lm_price_qual.fit()


# printing a summary of the results
print(results.summary())

<br>

In [ ]:
## Original Y and Log X ##

# INSTANTIATING a model type
lm_price_qual = smf.ols(formula = """Sale_Price ~ log_Lot_Area""",
                        data = housing)


# telling Python to FIT the data to the blueprint
results = lm_price_qual.fit()


# printing a summary of the results
print(results.summary())

<br>

In [ ]:
## Log Y and Log X ##

# INSTANTIATING a model type
lm_price_qual = smf.ols(formula = """log_Sale_Price ~ log_Lot_Area""",
                        data = housing)


# telling Python to FIT the data to the blueprint
results = lm_price_qual.fit()


# printing a summary of the results
print(results.summary())

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<h2>Team Challenge</h2>

<h4>Develop one optimal linear model for each version of the response variable (&nbsp;<em>Sale_Price</em> and <em>log_Sale_Price</em>&nbsp;).</h4>

* Use p-values as your feature significance criteria (&nbsp;<em>p</em> $\le$ 0.05&nbsp;).
* Optimize the model based on adjusted R-squared (<em>&nbsp;Adj. R-squared&nbsp;</em>).

In [ ]:
# preparing x-data
x_all = housing.drop([ 'Sale_Price', 'log_Sale_Price' ], axis = 1)


# preparing y-data
original_y = housing[ 'Sale_Price' ]
log_y      = housing[ 'log_Sale_Price' ]

<br>

In [ ]:
# checking documentation
help(quick_lm)

<br>

In [ ]:
## stepwise model using Sale_Price ##
sp_model = quick_lm(x_data        = _____,
                    y_data        = _____,
                    threshold_in  = 0.01,
                    threshold_out = 0.05,
                    test_size     = 0.25)

<br>

In [ ]:
## stepwise model using log_Sale_Price ##
log_sp_model = quick_lm(x_data        = _____,
                        y_data        = _____,
                        threshold_in  = 0.01,
                        threshold_out = 0.05,
                        test_size     = 0.25)

<br><br><hr style="height:.9px;border:none;color:#333;background-color:#333;" /><hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

~~~

  _   _ _                                             
 | \ | (_)                                            
 |  \| |_  ___ ___                                    
 | . ` | |/ __/ _ \                                   
 | |\  | | (_|  __/                                   
 |_| \_|_|\___\___|    _                              
 \ \        / /       | |                             
  \ \  /\  / /__  _ __| | __                          
   \ \/  \/ / _ \| '__| |/ /                          
    \  /\  / (_) | |  |   <                           
  ___\/_ \/ \___/|_|  |_|\_\_               _       _
 |  ____|                  | |             | |     | |
 | |____   _____ _ __ _   _| |__   ___   __| |_   _| |
 |  __\ \ / / _ \ '__| | | | '_ \ / _ \ / _` | | | | |
 | |___\ V /  __/ |  | |_| | |_) | (_) | (_| | |_| |_|
 |______\_/ \___|_|   \__, |_.__/ \___/ \__,_|\__, (_)
                       __/ |                   __/ |  
                      |___/                   |___/   
                                                                    
~~~

<br>
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Bonus: Additional Tools</h2>

Looking to speed up missing value imputations and skewness transformations? Check out <strong>simputer</strong> and <strong>transtorm</strong> in the preprocessing module of BaseRush!

In [ ]:
import baserush.preprocess as bp

<br>

In [ ]:
help(bp.simputer)

<br>

In [ ]:
help(bp.transtorm)

<br>